In [74]:
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
df_train = pd.read_csv(r'logs\inference\train_features.csv')
df_valid = pd.read_csv(r'logs\inference\valid_features.csv')

In [7]:
train = df_train.iloc[:,1:].values
valid = df_valid.iloc[:,1:].values

In [8]:
train.shape

(450, 512)

In [60]:
train_unit = np.array([vec/np.linalg.norm(vec) for vec in train])
mean_vector = np.mean(train_unit,axis=0)
mean_vector = mean_vector / np.linalg.norm(mean_vector)

In [64]:
def calc_anomalyDeg(x, mean_vector):
    x = x / np.linalg.norm(x)
    return 1 - np.dot(x,mean_vector)

In [68]:
train_anos = np.array([])
for t_vec in train_unit:
    train_anos = np.append(train_anos,calc_anomalyDeg(t_vec, mean_vector))

In [71]:
ano_m = np.mean(train_anos)
ano_S = np.mean(train_anos**2)

In [72]:
m_mo = (2*ano_m)/(ano_S - ano_m**2)
s_mo = (ano_S - ano_m**2)/(2 * ano_m)

In [73]:
m_mo, s_mo

(635.9847192611193, 0.001572364822163951)